## Start Node

In [1]:
import rclpy
from nimbro_utils.lazy import start_and_spin_node, stop_node

In [2]:
class MyNode(rclpy.node.Node):
    def __init__(self, context=None):
        super().__init__("test_parameter_handler_node", context=context)

In [3]:
node, executor, context, thread = start_and_spin_node(MyNode, blocking=False)

> Starting node 'MyNode'


## Add ParameterHandler

In [4]:
from nimbro_utils.lazy import ParameterHandler

In [5]:
node.parameter_handler = ParameterHandler(node)

#### The ParameterHandler's settings can be obtained via `get_settings()` and updated via `set_settings()`.

In [6]:
node.parameter_handler.get_settings()

{'severity': 20,
 'log_init_as_debug': True,
 'suffix': 'parameters',
 'parameters_container': 'parameters',
 'filter_callback': 'filter_parameter',
 'require_filter_callback': False}

In [7]:
node.parameter_handler.set_settings({'log_init_as_debug': False})

#### Parameter declarations and updates can be activated and deactivated.

In [8]:
node.parameter_handler.declarations_activated() # default is True

True

In [9]:
node.parameter_handler.deactivate_declarations()

2025-09-10 18:42:43.277 [INFO] [test_parameter_handler_node.parameters]: Declaring parameters deactivated


In [10]:
node.parameter_handler.activate_declarations()

2025-09-10 18:42:43.287 [INFO] [test_parameter_handler_node.parameters]: Declaring parameters activated


In [11]:
node.parameter_handler.updates_activated() # default is True

True

In [12]:
node.parameter_handler.deactivate_updates()

2025-09-10 18:42:43.313 [INFO] [test_parameter_handler_node.parameters]: Updating parameters deactivated


In [13]:
node.parameter_handler.activate_updates()

2025-09-10 18:42:43.323 [INFO] [test_parameter_handler_node.parameters]: Updating parameters activated


## Declare parameters

In [14]:
node.parameter_handler.declare(name="param_not_set",    dtype=None,        default_value=None)
node.parameter_handler.declare(name="param_bool",       dtype=bool,        default_value=False)
node.parameter_handler.declare(name="param_int",        dtype=int,         default_value=7, range_min=0, range_max=9, range_step=1, read_only=False) # default is read_only=True
node.parameter_handler.declare(name="param_float",      dtype=float,       default_value=4.2)
node.parameter_handler.declare(name="param_str",        dtype=str,         default_value="Hello", read_only=False)
node.parameter_handler.declare(name="param_list_bytes", dtype=list[bytes], default_value=[b'\x01', b'\x02', b'\xff', b'\x10'])
node.parameter_handler.declare(name="param_list_bool",  dtype=list[bool],  default_value=[True, False])
node.parameter_handler.declare(name="param_list_int",   dtype=list[int],   default_value=[1, 2, 3], read_only=False)
node.parameter_handler.declare(name="param_list_float", dtype=list[float], default_value=[3.1, 4.1, 5.9])
node.parameter_handler.declare(name="param_list_str",   dtype=list[str],   default_value=["H", "e", "l", "l", "o"])

2025-09-10 18:42:43.339 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_not_set' initialized to 'None'.
2025-09-10 18:42:43.342 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_bool' initialized to 'False'.
2025-09-10 18:42:43.345 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_int' initialized to '7'.
2025-09-10 18:42:43.348 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_float' initialized to '4.2'.
2025-09-10 18:42:43.350 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_str' initialized to 'Hello'.
2025-09-10 18:42:43.353 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_list_bytes' initialized to '[b'\x01', b'\x02', b'\xff', b'\x10']'.
2025-09-10 18:42:43.357 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_list_bool' initialized to '[True, False]'.
2025-09-10 18:42:43.361 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_list_int' initialized to 

## Access parameters

#### The name of Node attribute holding the parameter values ('parameters') is defined by the setting 'parameters_container'.

In [15]:
node.parameters.get()

{'param_not_set': None,
 'param_bool': False,
 'param_int': 7,
 'param_float': 4.2,
 'param_str': 'Hello',
 'param_list_bytes': [b'\x01', b'\x02', b'\xff', b'\x10'],
 'param_list_bool': [True, False],
 'param_list_int': [1, 2, 3],
 'param_list_float': [3.1, 4.1, 5.9],
 'param_list_str': ['H', 'e', 'l', 'l', 'o']}

In [16]:
"param_int" in node.parameters

True

In [17]:
node.parameters.param_int

7

In [18]:
node.parameters['param_int']

7

## Update parameters

In [19]:
node.parameter_handler.update(name="param_int", value="this is not an int")

2025-09-10 18:42:43.418 [ERROR] [test_parameter_handler_node.parameters]: Rejected attempt to update parameter 'param_int' to 'this is not an int': ValueError(Type 'Type.INTEGER' and value 'this is not an int' do not agree)


(False,
 "Rejected attempt to update parameter 'param_int' to 'this is not an int': ValueError(Type 'Type.INTEGER' and value 'this is not an int' do not agree)")

In [20]:
node.parameters.param_int

7

In [21]:
node.parameter_handler.update(name="param_int", value=20) # not in defined range

(False,
 "Rejected attempt to update parameter 'param_int' to '20': Value '20' is not in integer range '0' to '9'.")

2025-09-10 18:42:43.443 [ERROR] [test_parameter_handler_node.parameters]: Rejected attempt to update parameter 'param_int' to '20': Value '20' is not in integer range '0' to '9'.


In [22]:
node.parameters.param_int

7

In [23]:
node.parameter_handler.update(name="param_int", value=3)

(True, "Parameter 'param_int' set to '3'.")

2025-09-10 18:42:43.467 [INFO] [test_parameter_handler_node.parameters]: Parameter 'param_int' set to '3'.


In [24]:
node.parameters.param_int

3

## Reject and deflect parameter updates via callback

In [25]:
def filter_parameter(name, value, is_declared):
    message = None

    if name == "param_int":
        if value % 2 == 0:
            value += 1
            message = "Value must be uneven."

    if name == "param_str":
        valid_values = ["Hello", "World"]
        if value not in valid_values:
            message = f"Must be in {valid_values}."
            value = None

    return value, message

setattr(node, "filter_parameter", filter_parameter)

In [26]:
success, message = node.parameter_handler.update(name="param_int", value=8)

2025-09-10 18:42:43.498 [WARN] [test_parameter_handler_node.parameters]: Parameter 'param_int' deflected to '9': Value must be uneven.


In [27]:
print(success)
print(message)

True
Parameter 'param_int' deflected to '9' instead of '8'.


In [28]:
node.parameters.param_int

9

In [29]:
node.parameter_handler.update(name="param_str", value="test")

(False,
 "Parameter 'param_str' not set to 'test': Must be in ['Hello', 'World'].")

2025-09-10 18:42:43.526 [ERROR] [test_parameter_handler_node.parameters]: Parameter 'param_str' not set to 'test': Must be in ['Hello', 'World'].


In [30]:
node.parameters.param_str

'Hello'

## Undeclare parameters

In [31]:
node.parameter_handler.undeclare(name="param_int") # Read-only parameters cannot be undeclared.

2025-09-10 18:42:43.547 [INFO] [test_parameter_handler_node.parameters]: Successfully undeclared parameter 'param_int'.


In [32]:
"param_int" in node.parameters

False

## Stop Node

In [33]:
stop_node(node, executor, context, thread)

> Stopped node 'MyNode'
